In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import glob
import nibabel as nib
import os
import time

import pandas as pd
import numpy as np
import cv2
from skimage.transform import resize

from mricode.utils import log_textfile, createPath, data_generator
from mricode.utils import copy_colab
from mricode.utils import return_iter
from mricode.utils import return_csv
from mricode.config import config

from mricode.models.SimpleCNN_small import SimpleCNN
from mricode.models.DenseNet_NoDict import MyDenseNet

import tensorflow as tf
from tensorflow.keras.layers import Conv3D
from tensorflow import nn
from tensorflow.python.ops import nn_ops
from tensorflow.python.framework import tensor_shape
from tensorflow.python.keras.engine.base_layer import InputSpec
from tensorflow.python.keras.utils import conv_utils

tf.__version__

'2.0.0'

In [3]:
tf.test.is_gpu_available()

True

In [4]:
path_output = './output/'
path_tfrecords = '/data2/res64/down/'
path_csv = '/data2/csv/'
filename_res = {'train': 'intell_residual_train.csv', 'val': 'intell_residual_valid.csv', 'test': 'intell_residual_test.csv'}
filename_final = filename_res
sample_size = 'site16_allimages'
batch_size = 8
onlyt1 = False
modelname = 'densenet_allimages64'
Model = SimpleCNN
#Model = MyDenseNet
t1_mean=1.3779395849814497
t1_std=3.4895845243139503
t2_mean=2.22435586968901
t2_std=5.07708743178319
ad_mean=1.3008901218593748e-05
ad_std=0.009966655860940228
fa_mean=0.0037552628409334037
fa_std=0.012922319568740915
md_mean=9.827903909139596e-06
md_std=0.009956973204022659
rd_mean=8.237404999587111e-06
rd_std=0.009954672598675338

In [5]:
train_df, val_df, test_df, norm_dict = return_csv(path_csv, filename_final, False)

In [6]:
train_iter = config['down128']['iter_train']
val_iter = config['down128']['iter_val']
test_iter = config['down128']['iter_test']

In [7]:
if True:
  t1_mean = 0.
  t1_std = 0.
  t2_mean = 0.
  t2_std = 0.
  n = 0.
  for b in train_iter:
      log_textfile('meanstddown128_v2.log',str(n))
      t1_mean += np.mean(b['t1'])
      t1_std += np.std(b['t1'])
      t2_mean += np.mean(b['t2'])
      t2_std += np.std(b['t2'])
      n += np.asarray(b['t1']).shape[0]
      log_textfile('meanstddown128_v2.log',str((t1_mean/n, t1_std/n, t2_mean/n, t2_std/n)))

  t1_mean /= n
  t1_std /= n
  t2_mean /= n
  t2_std /= n

0.0
(1.4799309372901917, 3.615415361461672, 2.5243582725524902, 5.3110151290893555)
8.0
(1.480479370802641, 3.6190731859545435, 2.4764320850372314, 5.223186492919922)
16.0
(1.4837573344508808, 3.6339893830477017, 2.462623914082845, 5.213783899943034)
24.0
(1.4884509891271591, 3.612786063678251, 2.4468979239463806, 5.2129716873168945)
32.0
(1.4871513783931731, 3.611634891311982, 2.4254571914672853, 5.211108303070068)
40.0
(1.4682412122686703, 3.5973762442856874, 2.378411889076233, 5.189562082290649)
48.0
(1.4572635090776853, 3.6035214895362393, 2.3669818128858293, 5.187817709786551)
56.0
(1.4490590980276465, 3.6020146430625934, 2.3481189906597137, 5.166543245315552)
64.0
(1.4481266687313716, 3.587791773132633, 2.339432266023424, 5.1710327996148004)
72.0
(1.4373161517083646, 3.5880172022143113, 2.3246312856674196, 5.157265949249267)
80.0
(1.433799682693048, 3.585660150113169, 2.318388570438732, 5.147446849129417)
88.0
(1.446398905167977, 3.5831927715821705, 2.335479815800985, 5.169063806

784.0
(1.452138932591135, 3.5789421979586544, 2.344678770412098, 5.185190460898659)
792.0
(1.4521349601447582, 3.5807536913101528, 2.344109435081482, 5.190733742713928)
800.0
(1.4520543638137307, 3.580934513374533, 2.3453234941652505, 5.190685319428397)
808.0
(1.4511041718662954, 3.5799727432274153, 2.343979840185128, 5.19036434210983)
816.0
(1.4509555175177102, 3.5803493193150095, 2.344147404420723, 5.190282307782219)
824.0
(1.4506616267160728, 3.581129414269496, 2.3437572603042307, 5.188511142363915)
832.0
(1.4504901490041189, 3.5810994585478984, 2.3435552528926302, 5.187585730779738)
840.0
(1.4503810936409347, 3.5805555445898536, 2.342824731232985, 5.18731231959361)
848.0
(1.4501519783217216, 3.5813471812209525, 2.341873692574902, 5.1852441769894035)
856.0
(1.4505914053707212, 3.5814768108770783, 2.3418761248941773, 5.185601424287866)
864.0
(1.4510558635133122, 3.5808413951207876, 2.342512555078629, 5.187612026109608)
872.0
(1.4512980455024675, 3.580210877212544, 2.3429705944928254,

1552.0
(1.4557733435279283, 3.575271608091109, 2.3450409424610625, 5.18895060465886)
1560.0
(1.4562916451951071, 3.5756579315908885, 2.3457311160710392, 5.189605121709863)
1568.0
(1.455223168137715, 3.5757682174361385, 2.3445244561597174, 5.188725822468094)
1576.0
(1.4551031559780994, 3.5759451420091968, 2.344058729181386, 5.188024063303013)
1584.0
(1.45480855093829, 3.5757857992753226, 2.3438134409075406, 5.187471540729005)
1592.0
(1.454982534572482, 3.575786160529073, 2.3442342126369478, 5.188214406967163)
1600.0
(1.4550471532329992, 3.5761937414809757, 2.34417384418089, 5.1873498722095395)
1608.0
(1.4547630302829317, 3.5763069405590433, 2.3441641000237796, 5.187060020937778)
1616.0
(1.4548608748416596, 3.5758550237171502, 2.3444569886024365, 5.186853136335101)
1624.0
(1.454938110986761, 3.5762735158799304, 2.3447684213226916, 5.188175601117751)
1632.0
(1.455006108669246, 3.5765194613922957, 2.3450222864383603, 5.188215579056158)
1640.0
(1.4553784631843705, 3.5767838118914894, 2.3459

2320.0
(1.4594039698083376, 3.5728324062472523, 2.354429348227904, 5.195684567349883)
2328.0
(1.4593300624967438, 3.5728781622612744, 2.3542653526345343, 5.195900912154211)
2336.0
(1.4590935967481178, 3.572935296995393, 2.3541433241587044, 5.195726860098464)
2344.0
(1.458991878849714, 3.573195880812498, 2.3539588459494976, 5.195491185804614)
2352.0
(1.459013072994806, 3.5729435748770935, 2.353983205859944, 5.195529805199574)
2360.0
(1.4587660023590197, 3.572883548392221, 2.353712497530757, 5.195291706033655)
2368.0
(1.458688098523352, 3.5727446180514244, 2.3534109086701362, 5.195813562733557)
2376.0
(1.4587757691190948, 3.572725245849819, 2.353591346900735, 5.19666290923253)
2384.0
(1.458873123104955, 3.572530010919851, 2.3538910074776234, 5.197070110601725)
2392.0
(1.4585509428630272, 3.5723265174298264, 2.353578320344289, 5.1968415180842085)
2400.0
(1.458372335819113, 3.5726811691852465, 2.353529176046682, 5.1963730000974335)
2408.0
(1.4581875464330052, 3.5727816381712314, 2.35326723

3088.0
(1.4591901859302236, 3.5754965972097885, 2.3561014574627546, 5.198735100354335)
3096.0
(1.459386379890067, 3.5755281445511353, 2.3566079526832424, 5.199187407788542)
3104.0
(1.459273468693455, 3.5755662049592245, 2.3562951841820174, 5.198939609037267)
3112.0
(1.4593626451606934, 3.57550721135632, 2.356370586615342, 5.199224946437738)
3120.0
(1.459089334701638, 3.5753419164073152, 2.355727971362336, 5.198783112303985)
3128.0
(1.4590814241341181, 3.575295921052861, 2.3556949265149174, 5.198489636790995)
3136.0
(1.4590401368703854, 3.5753101748761162, 2.355414638689153, 5.19806396930879)
3144.0
(1.459118066724333, 3.575320012898303, 2.3562246270591234, 5.198809157773323)
3152.0
(1.4590699164550516, 3.5753371817741515, 2.356238415271421, 5.198776909067661)
3160.0
(1.4589072211146956, 3.575614576257075, 2.3559632030400364, 5.198035400323193)
3168.0
(1.45873756603495, 3.5756445098845417, 2.3555428147015705, 5.197927657542961)
3176.0
(1.458782040144331, 3.5754310320678817, 2.3558753782

3856.0
(1.4573946821023218, 3.573218468596861, 2.3539183988827848, 5.199319126689903)
3864.0
(1.4573942103754145, 3.5732275389281396, 2.3538869087361105, 5.199243161303938)
3872.0
(1.457482990346004, 3.573565888074722, 2.3541675159611652, 5.199637098410695)
3880.0
(1.4574574455158946, 3.5735082371117346, 2.3539796478954362, 5.199357079870907)
3888.0
(1.4575779790812204, 3.5735681918372433, 2.3539854554914594, 5.199514152088205)
3896.0
(1.457668639322529, 3.5738110144905137, 2.3539484148142766, 5.199560409686605)
3904.0
(1.457636362478777, 3.5737353111790857, 2.3539066924151713, 5.199597642465603)
3912.0
(1.4576965384033262, 3.5736991029571974, 2.354059831950129, 5.199901169173572)
3920.0
(1.4577280014844387, 3.573597949230712, 2.3541249719025403, 5.199853178439946)
3928.0
(1.4578211588739622, 3.5736541304377836, 2.3541697113494564, 5.199963475630535)
3936.0
(1.4576937493548916, 3.573470287103797, 2.354056778108856, 5.200034433639558)
3944.0
(1.4576028656621693, 3.573517980158252, 2.353

(1.4577774436362667, 3.5752673693090564, 2.353479934274018, 5.198319442531606)
4632.0
(1.457637735713145, 3.575305204704277, 2.3535294952063723, 5.198234956017856)
4640.0
(1.457649126277015, 3.575222068804601, 2.3536337854119465, 5.198194682700704)
4648.0
(1.4579252261169178, 3.575278137909614, 2.354031453837234, 5.198672692800305)
4656.0
(1.4577749422903126, 3.57533394870728, 2.3538215536605023, 5.198490778332509)
4664.0
(1.4576811051532015, 3.5752554981642883, 2.353689963686956, 5.198462091896632)
4672.0
(1.4576189406661906, 3.575269297521898, 2.3535326236333605, 5.198432840852656)
4680.0
(1.4577034756464966, 3.57518145632732, 2.3534685393649157, 5.198509353012762)
4688.0
(1.4577388759873795, 3.575246369944943, 2.3532550521766105, 5.1983655347937825)
4696.0
(1.4574717219133362, 3.575329151182461, 2.352744835574611, 5.1982653887093475)
4704.0
(1.4574500090602298, 3.5753340018771103, 2.3526262484098734, 5.197974792765438)
4712.0
(1.457437127828598, 3.5753720100606086, 2.352591321832042

5392.0
(1.457598821995435, 3.574139084606876, 2.3522969860500758, 5.1969646581013995)
5400.0
(1.4574910314125422, 3.57430920801132, 2.3522102603545556, 5.196718703360247)
5408.0
(1.4572857639902275, 3.574388861173436, 2.351818422094934, 5.196184240732799)
5416.0
(1.4572229611310987, 3.574395280361237, 2.351706448909456, 5.196124797725396)
5424.0
(1.4574145322695748, 3.5744596259969015, 2.3519346619214154, 5.196294725555735)
5432.0
(1.4574726568206269, 3.5745591276163564, 2.3520489576984853, 5.196092874162337)
5440.0
(1.4575371569058093, 3.57459680560101, 2.3521776391896205, 5.196242484402202)
5448.0
(1.457559346628329, 3.5746192699748724, 2.352317181150934, 5.196367517594369)
5456.0
(1.4574639374611145, 3.5745320635608477, 2.3520817854393976, 5.1961729830215475)
5464.0
(1.4574769551250322, 3.5744252024880447, 2.3520234474661756, 5.19614869669864)
5472.0
(1.4575085232183882, 3.57446227975337, 2.3521793483817666, 5.196294830489332)
5480.0
(1.457577031301253, 3.574457120425956, 2.35225237

In [25]:
%%time
counter = 0
for a in train_iter:
    counter+=1
    if counter == 10:
        break

KeyboardInterrupt: 

In [13]:
def return_iter(path, sample_size, batch_size=8, onlyt1=False, bl_cropped=False):
  # Some definitions
  if onlyt1:
    read_features = {
      't1': tf.io.FixedLenFeature([], dtype=tf.string),
      'subjectid': tf.io.FixedLenFeature([], dtype=tf.string)
    }
  else:
    read_features = {
      't1': tf.io.FixedLenFeature([], dtype=tf.string),
      't2': tf.io.FixedLenFeature([], dtype=tf.string),
      'subjectid': tf.io.FixedLenFeature([], dtype=tf.string)
    }
  def _parse_(serialized_example, decoder = np.vectorize(lambda x: x.decode('UTF-8')), onlyt1=False, bl_cropped=False):
    example = tf.io.parse_single_example(serialized_example, read_features)
    subjectid = example['subjectid']
    if not(onlyt1):
      if bl_cropped:
        t1 = tf.expand_dims(tf.reshape(tf.io.decode_raw(example['t1'], tf.float64), (128,128,128)), axis=-1)
        t2 = tf.expand_dims(tf.reshape(tf.io.decode_raw(example['t2'], tf.float32), (128,128,128)), axis=-1)
        return ({'t1': t1, 't2': t2, 'ad': '', 'fa': '', 'md': '', 'rd': '','subjectid': subjectid})
      else:
        t1 = tf.expand_dims(tf.reshape(tf.io.decode_raw(example['t1'], tf.int8), (128,128,128)), axis=-1)
        t2 = tf.expand_dims(tf.reshape(tf.io.decode_raw(example['t2'], tf.float32), (128,128,128)), axis=-1)
        return ({'t1': t1, 't2': t2, 'ad': '', 'fa': '', 'md': '', 'rd': '','subjectid': subjectid})
    else:
      t1 = tf.expand_dims(tf.reshape(tf.io.decode_raw(example['t1'], tf.int8), (256,256,256)), axis=-1)
      return ({'t1': t1, 'subjectid': subjectid})
  
  test_ds = tf.data.TFRecordDataset('/data2/t1t2_test_allimages_cropped_v4.tfrecords', num_parallel_reads=32)
  test_iter = test_ds.map(lambda x:_parse_(x, onlyt1=onlyt1, bl_cropped=bl_cropped), num_parallel_calls=32).batch(batch_size)
  return None, None, test_iter

In [14]:
train_iter, val_iter, test_iter = return_iter('/data2/res64/cropped/', 'allimages_cropped', 8, onlyt1=False, bl_cropped=True)

In [7]:
%%time
total_time = 0.
start_time = time.time()
for a in test_iter:
    total_time += time.time()-start_time
    print(time.time()-start_time)
    start_time = time.time()
    print(total_time)

1.464000940322876
1.4639976024627686
1.2421705722808838
2.7061643600463867
1.200373888015747
3.9065346717834473
1.2351112365722656
5.141641139984131
1.140338659286499
6.281976222991943
1.1556570529937744
7.4376280307769775
1.072843313217163
8.510468006134033
1.0454087257385254
9.555872201919556
1.151021957397461
10.706889867782593
1.009387731552124
11.716274738311768
1.2667765617370605
12.983038663864136
1.360337495803833
14.343372583389282
1.0754055976867676
15.418775081634521
1.4370663166046143
16.855836391448975
1.2051968574523926
18.0610294342041
1.3990216255187988
19.460047245025635
1.131829023361206
20.591872930526733
1.6224210262298584
22.214290380477905
1.5014653205871582
23.715751886367798
1.1664950847625732
24.882242918014526


KeyboardInterrupt: 

In [23]:
a = next(iter(test_iter))

StopIteration: 

In [19]:
a = next(val_iter)

In [10]:
test_df.shape[0]/8

156.125

In [11]:
for e in range(10):
    print(e)
    n = 0.
    for b in test_iter:
        n+=1
        print(n)

0
1.0
2.0
3.0
4.0
5.0
6.0
7.0
8.0
9.0
10.0
11.0
12.0
13.0
14.0
15.0
16.0
17.0
18.0
19.0
20.0
21.0
22.0
23.0
24.0
25.0
26.0
27.0
28.0
29.0
30.0
31.0
32.0
33.0
34.0
35.0
36.0
37.0
38.0
39.0
40.0
41.0
42.0
43.0
44.0
45.0
46.0
47.0
48.0
49.0
50.0
51.0
52.0
53.0
54.0
55.0
56.0
57.0
58.0
59.0
60.0
61.0
62.0
63.0
64.0
65.0
66.0
67.0
68.0
69.0
70.0
71.0
72.0
73.0
74.0
75.0
76.0
77.0
78.0
79.0
80.0
81.0
82.0
83.0
84.0
85.0
86.0
87.0
88.0
89.0
90.0
91.0
92.0
93.0
94.0
95.0
96.0
97.0
98.0
99.0
100.0
101.0
102.0
103.0
104.0
105.0
106.0
107.0
108.0
109.0
110.0
111.0
112.0
113.0
114.0
115.0
116.0
117.0
118.0
119.0
120.0
121.0
122.0
123.0
124.0
125.0
126.0
127.0
128.0
129.0
130.0
131.0
132.0
133.0
134.0
135.0
136.0
137.0
138.0
139.0
140.0
141.0
142.0
143.0
144.0
145.0
146.0
147.0
148.0
149.0
150.0
151.0
152.0
153.0
154.0
155.0
156.0
157.0
158.0
159.0
160.0
161.0
162.0
163.0
164.0
165.0
166.0
167.0
168.0
169.0


KeyboardInterrupt: 

In [13]:
if True:
  t1_mean = 0.
  t1_std = 0.
  t2_mean = 0.
  t2_std = 0.
  n = 0.
  for b in train_iter:
      log_textfile('meanstdcropped128.log',str(n))
      t1_mean += np.mean(b['t1'])
      t1_std += np.std(b['t1'])
      t2_mean += np.mean(b['t2'])
      t2_std += np.std(b['t2'])
      n += np.asarray(b['t1']).shape[0]
      log_textfile('meanstdcropped128.log',str((t1_mean/n, t1_std/n, t2_mean/n, t2_std/n)))

  t1_mean /= n
  t1_std /= n
  t2_mean /= n
  t2_std /= n

ZeroDivisionError: float division by zero

In [8]:
t1_mean, t1_std, t2_mean, t2_std

(26.53449535369873, 59.90857696533203, 40.260643005371094, 85.0711898803711)

In [9]:
log_textfile('meanstd256.log',str((t1_mean, t1_std, t2_mean, t2_std)))

(26.53449535369873, 59.90857696533203, 40.260643005371094, 85.0711898803711)


In [ ]:
error

In [10]:
if False:
  t1_mean = 0.
  t1_std = 0.
  t2_mean = 0.
  t2_std = 0.
  ad_mean = 0.
  ad_std = 0.
  fa_mean = 0.
  fa_std = 0.
  md_mean = 0.
  md_std = 0.
  rd_mean = 0.
  rd_std = 0.
  n = 0.
  for b in train_iter:
      t1_mean += np.mean(b['t1'])
      t1_std += np.std(b['t1'])
      t2_mean += np.mean(b['t2'])
      t2_std += np.std(b['t2'])
      a = np.asarray(b['ad'])
      a = a.copy()
      a[np.isnan(a)] = 0
      ad_mean += np.mean(a)
      ad_std += np.std(a)
      a = np.asarray(b['fa'])
      a = a.copy()
      a[np.isnan(a)] = 0
      fa_mean += np.mean(a)
      fa_std += np.std(a)
      a = np.asarray(b['md'])
      a = a.copy()
      a[np.isnan(a)] = 0
      md_mean += np.mean(a)
      md_std += np.std(a)
      a = np.asarray(b['rd'])
      a = a.copy()
      a[np.isnan(a)] = 0
      rd_mean += np.mean(a)
      rd_std += np.std(a)
      n += np.asarray(b['t1']).shape[0]

  t1_mean /= n
  t1_std /= n
  t2_mean /= n
  t2_std /= n
  ad_mean /= n
  ad_std /= n
  fa_mean /= n
  fa_std /= n
  md_mean /= n
  md_std /= n
  rd_mean /= n
  rd_std /= n

In [11]:
t1_mean, t1_std, t2_mean, t2_std, ad_mean, ad_std, fa_mean, fa_std, md_mean, md_std, rd_mean, rd_std

(1.3779395849814497,
 3.4895845243139503,
 2.22435586968901,
 5.07708743178319,
 1.3008901218593748e-05,
 0.009966655860940228,
 0.0037552628409334037,
 0.012922319568740915,
 9.827903909139596e-06,
 0.009956973204022659,
 8.237404999587111e-06,
 0.009954672598675338)

In [12]:
all_files = glob.glob('/data/home/benedikt_d_schifferer/data_T2_lowerres_cropped128/' + '*.gz') 
all_files += glob.glob('/data/home/benedikt_d_schifferer/data_T1_lowerres_cropped128/' + '*.gz')
all_files = glob.glob('/data/home/benedikt_d_schifferer/data_T1_T2_201909/' + '*.gz') 

train_iter = data_generator(list(train_df['subjectkey'].values), all_files, batch_size, 
                            input_shape=(256,256,256))
val_iter = data_generator(list(val_df['subjectkey'].values), all_files, batch_size, 
                            is_validation_data=True, 
                            input_shape=(256,256,256))
test_iter = data_generator(list(test_df['subjectkey'].values), all_files, batch_size, 
                            is_validation_data=True, 
                            input_shape=(256,256,256))

In [14]:
if True:
  t1_mean = 0.
  t1_std = 0.
  t2_mean = 0.
  t2_std = 0.
  n = 0.
  for b in train_iter:
      t1_mean += np.mean(b['t1'])
      t1_std += np.std(b['t1'])
      t2_mean += np.mean(b['t2'])
      t2_std += np.std(b['t2'])
      n += np.asarray(b['t1']).shape[0]
      print(n)

  t1_mean /= n
  t1_std /= n
  t2_mean /= n
  t2_std /= n

8.0
16.0
24.0


KeyboardInterrupt: 

In [ ]:
error

In [13]:
norm_dict

{'BMI': {'mean': 18.681548127052135, 'std': 4.193043131845343},
 'age': {'mean': 119.00325844623563, 'std': 7.479129774017182},
 'height': {'mean': 55.25173666322929, 'std': 3.152756181679028},
 'nihtbx_cardsort_uncorrected': {'mean': 0.01902727147573316,
  'std': 0.92710655806542},
 'nihtbx_cryst_uncorrected': {'mean': 0.007018628014748754,
  'std': 0.7845373584638602},
 'nihtbx_flanker_uncorrected': {'mean': 0.02188780794049048,
  'std': 0.9070917080607726},
 'nihtbx_fluidcomp_uncorrected': {'mean': 0.020178243913565427,
  'std': 0.86606123778624},
 'nihtbx_list_uncorrected': {'mean': 0.00625176016120734,
  'std': 0.8898550695735616},
 'nihtbx_pattern_uncorrected': {'mean': 0.020721412569885096,
  'std': 0.9486618556882954},
 'nihtbx_picture_uncorrected': {'mean': 0.0005782175223803825,
  'std': 0.9577989703304521},
 'nihtbx_picvocab_uncorrected': {'mean': 0.0068509109986280275,
  'std': 0.8038465951211212},
 'nihtbx_reading_uncorrected': {'mean': 0.0027847502208883574,
  'std': 0.85

In [11]:
cat_cols = {'female': 2, 'race.ethnicity': 5, 'high.educ_group': 4, 'income_group': 8, 'married': 6}
num_cols = [x for x in list(val_df.columns) if '_norm' in x]

In [12]:
def calc_loss_acc(out_loss, out_acc, y_true, y_pred, cat_cols, num_cols, norm_dict):
  for col in num_cols:
    tmp_col = col
    tmp_std = norm_dict[tmp_col.replace('_norm','')]['std']
    tmp_y_true = tf.cast(y_true[col], tf.float32).numpy()
    tmp_y_pred = np.squeeze(y_pred[col].numpy())
    if not(tmp_col in out_loss):
      out_loss[tmp_col] = np.sum(np.square(tmp_y_true-tmp_y_pred))
    else:
      out_loss[tmp_col] += np.sum(np.square(tmp_y_true-tmp_y_pred))
    if not(tmp_col in out_acc):
      out_acc[tmp_col] = np.sum(np.square((tmp_y_true-tmp_y_pred)*tmp_std))
    else:
      out_acc[tmp_col] += np.sum(np.square((tmp_y_true-tmp_y_pred)*tmp_std))
  for col in list(cat_cols.keys()):
    tmp_col = col
    if not(tmp_col in out_loss):
      out_loss[tmp_col] = tf.keras.losses.SparseCategoricalCrossentropy()(tf.squeeze(y_true[col]), tf.squeeze(y_pred[col])).numpy()
    else:
      out_loss[tmp_col] += tf.keras.losses.SparseCategoricalCrossentropy()(tf.squeeze(y_true[col]), tf.squeeze(y_pred[col])).numpy()
    if not(tmp_col in out_acc):
      out_acc[tmp_col] = tf.reduce_sum(tf.dtypes.cast((y_true[col] == tf.argmax(y_pred[col], axis=-1)), tf.float32)).numpy()
    else:
      out_acc[tmp_col] += tf.reduce_sum(tf.dtypes.cast((y_true[col] == tf.argmax(y_pred[col], axis=-1)), tf.float32)).numpy()    
  return(out_loss, out_acc)

def format_output(out_loss, out_acc, n, cols, print_bl=False):
  loss = 0
  acc = 0
  output = []
  for col in cols:
    output.append([col, out_loss[col]/n, out_acc[col]/n])
    loss += out_loss[col]/n
    acc += out_acc[col]/n
  df = pd.DataFrame(output)
  df.columns = ['name', 'loss', 'acc']
  if print_bl:
    print(df)
  return(loss, acc, df)

@tf.function
def train_step(X, y, model, optimizer, cat_cols, num_cols):
  with tf.GradientTape() as tape:
    predictions = model(X)
    i = 0
    loss = tf.keras.losses.MSE(tf.cast(y[num_cols[i]], tf.float32), tf.squeeze(predictions[num_cols[i]]))
    for i in range(1,len(num_cols)):
      loss += tf.keras.losses.MSE(tf.cast(y[num_cols[i]], tf.float32), tf.squeeze(predictions[num_cols[i]]))
    for col in list(cat_cols.keys()):
      loss += tf.keras.losses.SparseCategoricalCrossentropy()(tf.squeeze(y[col]), tf.squeeze(predictions[col]))
  gradients = tape.gradient(loss, model.trainable_variables)
  mean_std = [x.name for x in model.non_trainable_variables if ('batch_norm') in x.name and ('mean' in x.name or 'variance' in x.name)]
  with tf.control_dependencies(mean_std):
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
  return(y, predictions, loss)

@tf.function
def test_step(X, y, model):
  predictions = model(X)
  return(y, predictions)

def epoch(data_iter, df, model, optimizer, cat_cols, num_cols, norm_dict):
  out_loss = {}
  out_acc = {}
  n = 0.
  n_batch = 0.
  total_time_dataload = 0.
  total_time_model = 0.
  start_time = time.time()
  for batch in data_iter:
    total_time_dataload += time.time() - start_time
    start_time = time.time()
    t1 = (tf.cast(batch['t1'], tf.float32)-t1_mean)/t1_std
    t2 = (batch['t2']-t2_mean)/t2_std
    if False:
        ad = batch['ad']
        ad = tf.where(tf.math.is_nan(ad), tf.zeros_like(ad), ad)
        ad = (ad-ad_mean)/ad_std
        fa = batch['fa']
        fa = tf.where(tf.math.is_nan(fa), tf.zeros_like(fa), fa)
        fa = (fa-fa_mean)/fa_std
        md = batch['md']
        md = tf.where(tf.math.is_nan(md), tf.zeros_like(md), md)
        md = (md-md_mean)/md_std
        rd = batch['rd']
        rd = tf.where(tf.math.is_nan(rd), tf.zeros_like(rd), rd)
        rd = (rd-rd_mean)/rd_std
    subjectid = decoder(batch['subjectid'])
    y = get_labels(df, subjectid, list(cat_cols.keys())+num_cols)
    #X = tf.concat([t1], axis=4)
    X = tf.concat([t1, t2], axis=4)
    if optimizer != None:
      y_true, y_pred, loss = train_step(X, y, model, optimizer, cat_cols, num_cols)
    else:
      y_true, y_pred = test_step(X, y, model)
    out_loss, out_acc = calc_loss_acc(out_loss, out_acc, y_true, y_pred, cat_cols, num_cols, norm_dict)
    n += X.shape[0]
    n_batch += 1
    if (n_batch % 10) == 0:
      print(n_batch)
    total_time_model += time.time() - start_time
    start_time = time.time()
  return (out_loss, out_acc, n, total_time_model, total_time_dataload)

def get_labels(df, subjectid, cols = ['nihtbx_fluidcomp_uncorrected_norm']):
  subjects_df = pd.DataFrame(subjectid)
  result_df = pd.merge(subjects_df, df, left_on=0, right_on='subjectkey', how='left')
  output = {}
  for col in cols:
    output[col] = np.asarray(result_df[col].values)
  return output

def best_val(df_best, df_val, df_test, e):
  df_best = pd.merge(df_best, df_val, how='left', left_on='name', right_on='name')
  df_best = pd.merge(df_best, df_test, how='left', left_on='name', right_on='name')
  df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'best_loss_epochs'] = e
  df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_epochs'] = e
  df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'best_loss_test'] = df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'cur_loss_test']
  df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'best_loss_val'] = df_best.loc[df_best['best_loss_val']>=df_best['cur_loss_val'], 'cur_loss_val']
  df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_test'] = df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'cur_acc_test']
  df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_val'] = df_best.loc[(df_best['best_acc_val']<=df_best['cur_acc_val'])&(df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'cur_acc_val']
  df_best.loc[(df_best['best_acc_val']>=df_best['cur_acc_val'])&(~df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_test'] = df_best.loc[(df_best['best_acc_val']>=df_best['cur_acc_val'])&(~df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'cur_acc_test']
  df_best.loc[(df_best['best_acc_val']>=df_best['cur_acc_val'])&(~df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'best_acc_val'] = df_best.loc[(df_best['best_acc_val']>=df_best['cur_acc_val'])&(~df_best['name'].isin(['female', 'race.ethnicity', 'high.educ_group', 'income_group', 'married'])), 'cur_acc_val']
  df_best = df_best.drop(['cur_loss_val', 'cur_acc_val', 'cur_loss_test', 'cur_acc_test'], axis=1)
  return(df_best)

In [13]:
decoder = np.vectorize(lambda x: x.decode('UTF-8'))
template = 'Epoch {0}, Loss: {1:.3f}, Accuracy: {2:.3f}, Val Loss: {3:.3f}, Val Accuracy: {4:.3f}, Time Model: {5:.3f}, Time Data: {6:.3f}'
for col in [0]:
  log_textfile(path_output + modelname + '/log' + '.log', cat_cols),
  log_textfile(path_output + modelname + '/log' + '.log', num_cols)
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
  optimizer = tf.keras.optimizers.Adam(lr = 0.001)
  model = Model(cat_cols, num_cols)
  df_best = None
  for e in range(20):
    log_textfile(path_output + modelname + '/log' + '.log', 'Epochs: ' + str(e))
    loss = tf.Variable(0.)
    acc = tf.Variable(0.) 
    val_loss = tf.Variable(0.)
    val_acc = tf.Variable(0.)
    test_loss = tf.Variable(0.)
    test_acc = tf.Variable(0.)
    tf.keras.backend.set_learning_phase(True)
    train_out_loss, train_out_acc, n, time_model, time_data = epoch(train_iter, train_df, model, optimizer, cat_cols, num_cols, norm_dict)
    tf.keras.backend.set_learning_phase(False)
    val_out_loss, val_out_acc, n, _, _ = epoch(val_iter, val_df, model, None, cat_cols, num_cols, norm_dict)
    test_out_loss, test_out_acc, n, _, _ = epoch(test_iter, test_df, model, None, cat_cols, num_cols, norm_dict)
    loss, acc, _ = format_output(train_out_loss, train_out_acc, n, list(cat_cols.keys())+num_cols)
    val_loss, val_acc, df_val = format_output(val_out_loss, val_out_acc, n, list(cat_cols.keys())+num_cols, print_bl=False)
    test_loss, test_acc, df_test = format_output(test_out_loss, test_out_acc, n, list(cat_cols.keys())+num_cols, print_bl=False)
    df_val.columns = ['name', 'cur_loss_val', 'cur_acc_val']
    df_test.columns = ['name', 'cur_loss_test', 'cur_acc_test']
    if e == 0:
      df_best = pd.merge(df_test, df_val, how='left', left_on='name', right_on='name')
      df_best['best_acc_epochs'] = 0
      df_best['best_loss_epochs'] = 0
      df_best.columns = ['name', 'best_loss_test', 'best_acc_test', 'best_loss_val', 'best_acc_val', 'best_acc_epochs', 'best_loss_epochs']
    df_best = best_val(df_best, df_val, df_test, e)
    print(df_best[['name', 'best_loss_test', 'best_acc_test']])
    print(df_best[['name', 'best_loss_val', 'best_acc_val']])
    log_textfile(path_output + modelname + '/log' + '.log', template.format(e, loss, acc, val_loss, val_acc, time_model, time_data))
    if e in [10, 15]:
      optimizer.lr = optimizer.lr/3
      log_textfile(path_output + modelname + '/log' + '.log', 'Learning rate: ' + str(optimizer.lr))
    df_best.to_csv(path_output +  modelname + '/df_best' + str(e) + '.csv')
    df_best.to_csv(path_output +  modelname + '/df_best' + '.csv')
    model.save_weights(path_output + modelname + '/checkpoints/' + str(e) + '/')

{'high.educ_group': 4, 'married': 6, 'income_group': 8, 'race.ethnicity': 5, 'female': 2}
['BMI_norm', 'age_norm', 'vol_norm', 'weight_norm', 'height_norm', 'nihtbx_fluidcomp_uncorrected_norm', 'nihtbx_cryst_uncorrected_norm', 'nihtbx_pattern_uncorrected_norm', 'nihtbx_picture_uncorrected_norm', 'nihtbx_list_uncorrected_norm', 'nihtbx_flanker_uncorrected_norm', 'nihtbx_picvocab_uncorrected_norm', 'nihtbx_cardsort_uncorrected_norm', 'nihtbx_totalcomp_uncorrected_norm', 'nihtbx_reading_uncorrected_norm']
Epochs: 0
10.0


KeyboardInterrupt: 

In [ ]:
error

In [13]:
test_loss, test_acc, df_test = format_output(test_out_loss, test_out_acc, n, list(cat_cols.keys())+num_cols, print_bl=False)

In [15]:
df_test.to_csv('final_output_all.csv')

In [17]:
inputs = tf.keras.Input(shape=(64,64,64,2), name='inputlayer123')
a = model(inputs)['female']
mm = tf.keras.models.Model(inputs=inputs, outputs=a)

In [18]:
from tf_explain.core.smoothgrad import SmoothGrad
import pickle

explainer = SmoothGrad()
output_grid = {}
output_n = {}
for i in range(2):
  output_grid[i] = np.zeros((64,64,64))
  output_n[i] = 0
counter = 0
for batch in test_iter:
  counter+=1
  print(counter)
  t1 = (tf.cast(batch['t1'], tf.float32)-t1_mean)/t1_std
  t2 = (batch['t2']-t2_mean)/t2_std
  X = tf.concat([t1, t2], axis=4)
  subjectid = decoder(batch['subjectid'])
  y = get_labels(test_df, subjectid, list(cat_cols.keys())+num_cols)
  y_list = list(y['female'])
  for i in range(X.shape[0]):
    X_i = X[i]
    X_i = tf.expand_dims(X_i, axis=0)
    y_i = y_list[i]
    grid = explainer.explain((X_i, _), mm, y_i, 20, 1.)
    output_grid[y_i] += grid
    output_n[y_i] += 1

1
2
3


KeyboardInterrupt: 

In [19]:
pickle.dump([output_grid, output_n], open( "smoothgrad_female_all.p", "wb" ) )

In [21]:
#output_grid, output_n = pickle.load(open( "smoothgrad_female.p", "rb" ))

In [25]:
def apply_grey_patch(image, top_left_x, top_left_y, top_left_z, patch_size):
    """
    Replace a part of the image with a grey patch.
    Args:
        image (numpy.ndarray): Input image
        top_left_x (int): Top Left X position of the applied box
        top_left_y (int): Top Left Y position of the applied box
        patch_size (int): Size of patch to apply
    Returns:
        numpy.ndarray: Patched image
    """
    patched_image = np.array(image, copy=True)
    patched_image[
        top_left_x : top_left_x + patch_size, top_left_y : top_left_y + patch_size, top_left_z : top_left_z + patch_size, :
    ] = 0

    return patched_image

In [26]:
import math

In [29]:
def get_sensgrid(image, mm, class_index, patch_size):
  sensitivity_map = np.zeros((
    math.ceil(image.shape[0] / patch_size),
    math.ceil(image.shape[1] / patch_size),
    math.ceil(image.shape[2] / patch_size)
  ))
  for index_z, top_left_z in enumerate(range(0, image.shape[2], patch_size)):
    patches = [
              apply_grey_patch(image, top_left_x, top_left_y, top_left_z, patch_size)
              for index_x, top_left_x in enumerate(range(0, image.shape[0], patch_size))
              for index_y, top_left_y in enumerate(range(0, image.shape[1], patch_size))
              ]
    coordinates = [
                (index_y, index_x)
                for index_x, _ in enumerate(range(0, image.shape[0], patch_size))
                for index_y, _ in enumerate(range(0, image.shape[1], patch_size))
                ]
    predictions = mm.predict(np.array(patches), batch_size=1)
    target_class_predictions = [prediction[class_index] for prediction in predictions]
    for (index_y, index_x), confidence in zip(coordinates, target_class_predictions):
      sensitivity_map[index_y, index_x, index_z] = 1 - confidence
  sm = resize(sensitivity_map, (64,64,64))
  heatmap = (sm - np.min(sm)) / (sm.max() - sm.min())
  return(heatmap)

In [28]:
output_grid = {}
output_n = {}
for i in range(2):
  output_grid[i] = np.zeros((64,64,64))
  output_n[i] = 0

counter = 0
for batch in test_iter:
  counter+=1
  print(counter)
  t1 = (tf.cast(batch['t1'], tf.float32)-t1_mean)/t1_std
  t2 = (batch['t2']-t2_mean)/t2_std
  X = tf.concat([t1, t2], axis=4)
  subjectid = decoder(batch['subjectid'])
  y = get_labels(test_df, subjectid, list(cat_cols.keys())+num_cols)
  y_list = list(y['female'])
  for i in range(X.shape[0]):
    print(i)
    X_i = X[i]
    y_i = y_list[i]
    grid = get_sensgrid(X_i, mm, y_i, 4)
    output_grid[y_i] += grid
    output_n[y_i] += 1
  if counter==6:
    break

1
0


NameError: name 'resize' is not defined

In [ ]:
pickle.dump([output_grid, output_n], open( "heatmap_female_all.p", "wb" ) )

In [13]:
error

NameError: name 'error' is not defined

In [ ]:
batch = next(iter(train_iter))

In [ ]:
t1 = (tf.cast(batch['t1'], tf.float32)-t1_mean)/t1_std
t2 = (batch['t2']-t2_mean)/t2_std
ad = batch['ad']
ad = tf.where(tf.math.is_nan(ad), tf.zeros_like(ad), ad)
ad = (ad-ad_mean)/ad_std
fa = batch['fa']
fa = tf.where(tf.math.is_nan(fa), tf.zeros_like(fa), fa)
fa = (fa-fa_mean)/fa_std
md = batch['md']
md = tf.where(tf.math.is_nan(md), tf.zeros_like(md), md)
md = (md-md_mean)/md_std
rd = batch['rd']
rd = tf.where(tf.math.is_nan(rd), tf.zeros_like(rd), rd)
rd = (rd-rd_mean)/rd_std
#subjectid = decoder(batch['subjectid'])
#y = get_labels(df, subjectid, list(cat_cols.keys())+num_cols)
#X = tf.concat([t1, t2, ad, fa, md, rd], axis=4)
X = tf.concat([t1, t2], axis=4)    

In [ ]:
tf.keras.backend.set_learning_phase(True)
model(X)['female']

In [ ]:
tf.keras.backend.set_learning_phase(False)
model(X)['female']

In [ ]:
mean_std = [x.name for x in model.non_trainable_variables if ('batch_norm') in x.name and ('mean' in x.name or 'variance' in x.name)]

In [ ]:
model = Model(cat_cols, num_cols)

In [ ]:
model.non_trainable_variables 